# Model Exploration
In this file, I will be investigating the PAWS model for Stackelberg Security Games and Wildlife Conservation: https://projects.iq.harvard.edu/files/teamcore/files/project_paws_publication_01_teamcore_yang_et_al_2014.pdf. In this paper, the authors design a stackelberg game where the two agents are the rangers and the poachers. They play a turn based game where the rangers first setup a patrol strategy. The poachers can then observe this strategy and come up with a poaching strategy in return. The patrol/poaching areas are discritized by splitting the map into a grid, and a strategy is defined by assigning the limited number of rangers to a cell of the grid. From here, utility is assigned to the different outcomes of people present combined with if there are endangered animals present, with positive utility given to catching and stopping poachers and negative utility to poachers without a ranger. Using this score of the board, the rangers then update their strategy to reflect the observed poacher behavior. This cycle repeats with the goal of rangers encountering more poachers.

For our approach, we want to use this same model with the agents being the trappers and the invasive species we are looking to capture. To make this modification, we first need to change the utility structure such that we reward traps capturing the invasive species and disincentivize traps being placed somewhere where they dont catch an invasive animal. To do this, we first need to understand how the original paws implementation works.

## PAWS Structural Outline and the modifications we will need to make

From my 91r work junior fall, I have put together a rough outline of the PAWS codebase as it works as a flask app in python. To run the model, researchers do the following things:

1. **Setup the Flask App and log**: Sets up the overhead and logging needed to track the execution of the flask app containing this software. This portion also establishes connection to an Azure server containing the data needed to operate the program. This portion, relevant for deployment and scaling, will not be relevant to our implementation.
2. **Data Input and Breakdown**: In the PAWS design, researchers take data keys from a JSON file that then allow them to access the data of interest from the cloud. This data comes in 3 parts:
    - shapes, or the shapefiles representing the patrolled park (i.e. park boundaries or locations of different features of the landscape like rivers and roads)
    - rasters, or the raster files providing supplemental information about a patrolled park (i.e. elevation of different areas in a park, land cover)
    - patrol_observations, or a csv containing information about the different patrols used for prediction.
    
    I have a data dictionary for both the JSON file and patrol_observations file to understand what data goes where. Generally though, for our implementation we will be cutting the JSON mapping to the cloud and grabbing the right datasets and instead creating a function to setup the data we need locally.

3. **Data Validation and Error Handling**: this section allows us to verify that necessary features of a given piece of data holds before attempting to process the data for prediction. Problems are broken down into Warnings, Errors, and FatalErrors that will log and kill the program if fatal. Understanding and updating these checks based on the modified reward and data will be important as we test and develop our modifications.
4. **Preprocess**: This does all of the data pre-processing, where we grab the boundary, clean the data, get shapefiles, distances, and rasters, calculate effort, and compute illegal activity. Some data verification does happen in this file, so checking if this data verification can be refactored into the data validation section could be useful. For our implementation, I think the preprocessing will look quite similar with the exception of how to process the reward-defining activity.
5. **Consolidate**: This consolidates all of the data pieces into a single dataframe for the model to use. The PAWS implementaton of consolidate contains some repeated processes that already exist in preprocess, so for our implementation it will be nice to remove those repeat processes. Besides this, consolidation should look extremely similar for our implementation.
6. **Predict**: Here, taking this single dataframe, we ultimately predict the locations for our next round of patrols. For our implementation, the prediction should be identical, so long as the modified reward is captured within the data. If not, then the actual prediction schematic may need to update its scoring to reflect rewarding the successful traps. 

## Invasive Species Modifications